### A script to retrieve FASTA header/sequence pairs

`sys.argv` is a list of values from the command line.

The first element of this list, `sys.argv[0]`, is the script itself. Consider the following example:

`python3 my_script.py alpaca baboon camel`

In this case, `sys.argv` would be:

In [2]:
sys_args = ['path_to_my_script.py', 'alpaca', 'baboon', 'camel']

This list can then be indexed as expected:

In [6]:
print(sys_args[0])
print(sys_args[1])

path_to_my_script.py
alpaca


If we want all of the arguments supplied to the script as a list, we can just do

In [8]:
args = sys_args[1:]
args

['alpaca', 'baboon', 'camel']

So, for our script we want the user to be able to provide one or more header values for the script to retrieve from the file. We will require that the first argument (`sys.argv[1]`) be the FASTA file, and any following arguments will be headers:

In [10]:
import sys  # We need sys to get input from the command line

def fasta_parse(fasta, delimiter=">", separator="", trim_header=True):
    """
    Iterator which takes FASTA as input. Yields
    header/value pairs. Separator will be
    used to join the return value; use separator=
    None to return a list.

    If trim_header, parser will return the
    FASTA header up to the first space character.
    Otherwise, it will return the full, unaltered
    header string.

    """
    header, seq = None, []
    with open(fasta) as f:
        for line in f:
            if line.startswith(delimiter):
                if header:  # associate accumulated seq with header
                    if separator is not None:
                        seq = separator.join(str(e) for e in seq)
                    yield header, seq
                # Assign a new header
                header = line.strip().lstrip(delimiter)
                if trim_header:
                    header = header.split()[0]
                # Clear seq for new round of collection
                seq = []
            elif line.startswith('#'):
                continue
            else:
                if line.strip():  # don't collect blank lines
                    seq.append(line.rstrip('\n'))
        if separator is not None:  # make string
            seq = separator.join(str(e) for e in seq)
        yield header, seq

fasta_file = sys.argv[1]
headers = sys.argv[2:]  # every additional argument, as a list
headers = set(headers)  # no duplicate headers

header_n_wanted = len(headers)
found_header_count = 0

# We have our fasta_parse function, which gives ('yields') every header/sequence pair 
# in a FASTA file as an iterable (which means we can loop over its values)
for header, sequence in fasta_parse(fasta_file):  # the function opens the file internally
    # check if each header is wanted by the user
    if header in headers:  # print the header and its sequence
        found_header_count += 1
        print('>' + header)
#         print('>{}'.format(header))
        print(sequence)
        headers.remove(header)
        # exit the loop as early as possible so we aren't looping through
        # the whole file if we don't need to
        if len(headers) == 0:  # all headers found
            break  # exit the containing loop
            
# Give user some info about how things went;
# the `file` argument allows for the output to be sent to a different
# stream so that it's not captured in any redirection the user might
# make
print('# Found {}/{} headers'.format(found_header_count, header_n_wanted), file=sys.stderr)

sys.exit(0)  # a formality